In [1]:
!pip install -U kaggle-environments
from kaggle_environments import make, evaluate
import numpy as np
import torch
import torch.nn.functional as F
from random import choice
print(np.__version__, torch.__version__)

In [0]:
# Constants #

# State #
init, observe = False, None


# Map #
class Direction:
    NORTH = 1
    SOUTH = 2
    EAST = 3
    WEST = 4
    STATIONARY = 5


class Position:
    def __init__(self, x, y, halite=0, ship=None, shipyard=None):
        self.x, self.y, self.halite, self.ship, self.shipyard = x, y, halite, ship, shipyard


# Units #
class Ship:
    def __init__(self, unitID, x, y):
        self.unitID = unitID
        self.x = x
        self.y = y


class Shipyard:
    def __init__(self, unitID, x, y):
        self.unitID = unitID
        self.x = x
        self.y = y


# Observe #
class Observation:
    def __init__(self, config, obs):
        self.CFG = CFG
        self.SIZE = self.config.size
        self.ME = obs.player
        self.PLAYER_N = len(obs.players)

        self.ships = [{} for x in range(len(obs.players))]
        self.shipyards = [{} for x in range(len(obs.players))]
        self.haliteStorage = [0 for x in range(len(obs.players))]
        self.haliteMap = np.empty([self.SIZE, self.SIZE])
        self.board = [[Position(x, y) for x in range(self.SIZE)]
                      for y in range(self.SIZE)]

    def update(self, obs):
        self.PLAYER_N = len(obs.players)
        self.turn += 1

        # Update halite info
        self.haliteMap = np.reshape(
            np.float32(obs.halite), self.haliteMap.shape)
        for x in range(self.CFG.size):
            for y in range(self.CFG.size):
                self.board[x][y].halite = self.haliteMap[x][y]

        # Update player specific info
        raw = obs.players
        for player in range(len(obs.players)):
            self.haliteStorage[player] = obs.players[player][0]
            shipyards = obs.players[player][1]
            ships = obs.players[player][2]
            # Updating state info for seperate units
            tempShips = {}
            tempShipyards = {}
            # Shipyards
            for shipyardID in shipyards.keys():
                x, y = self.decode_pos(shipyards[shipyardID])
                self.board[x][y].shipyard = shipyardID
                if shipyardID in self.shipyards[player]:
                    self.shipyards[player][shipyardID].x = x
                    self.shipyards[player][shipyardID].x = y
                    tempShipyards[shipyardID] = self.shipyards[player][
                        shipyardID]
                else:
                    tempShipyards[shipyardID] = Shipyard(shipyardID, x, y)
            self.shipyards[player] = tempShipyards
            # Ships
            for shipID in ships.keys():
                x, y = self.decode_pos(ships[shipID][0])
                self.board[x][y].ship = shipID
                if shipID in self.ships[player]:
                    self.ships[player][shipID].x = x
                    self.ships[player][shipID].x = y
                    tempShips[shipID] = self.ships[player][shipID]
                else:
                    tempShips[shipID] = Ship(shipID, x, y)
                tempShips[shipID].halite = ships[shipID][1]
            self.ships[player] = tempShips

    def decode_pos(self, n):
        return n % self.SIZE, n // self.SIZE

# Tool #
def dryMove(pos, direction):
    if direction == "NORTH":
        return ((pos[0] - 1) % size, pos[1])
    elif direction == "SOUTH":
        return ((pos[0] + 1) % size, pos[1])
    elif direction == "EAST":
        return (pos[0], (pos[1] + 1) % size)
    elif direction == "WEST":
        return (pos[0], (pos[1] - 1) % size)
    else:  # STATIONARY
        return pos
    
def distance_to(self, target):
    if type(target) is Position:
        col = abs(self.x - target.x)
        col = min(col, observe.SIZE - col)
        row = abs(self.y - target.y)
        row = min(row_d, observe.SIZE - row)
        return row + col

def direction_to(self, target):
    res = Direction.STATIONARY
    if target.x != self.x:
        if target.x > self.x and target.x - self.x < observe.SIZE / 2:
            res = Direction.EAST
        elif self.x > target.x and self.x - target.x > observe.SIZE / 2:
            res = Direction.EAST
        else:
            res = Direction.WEST
    elif target.y != self.y:
        if target.y > self.y and target.y - self.y < observe.SIZE / 2:
            res = Direction.NORTH
        elif self.y > target.y and self.y - target.y > observe.SIZE / 2:
            res = Direction.NORTH
        else:
            res = Direction.SOUTH
    return res


In [0]:
def agent(obs, config):
    global init, observe

    action = {}
    
    # init
    if not init:
        observe = Observation(config,obs)
        init = True

    # parse obs
    observe.update(obs)

    # decide

    return action

# Gameplay

In [0]:
env = make("halite", debug=True)
env.run([agent, "random"])
env.render(mode="ipython", width=800, height=600)

In [0]:
import json
print("Configuration:", json.dumps(env.specification.configuration, indent=4, sort_keys=True))
print("Observation:", json.dumps(env.specification.observation, indent=4, sort_keys=True))
print("Action:", json.dumps(env.specification.action, indent=4, sort_keys=True))